# Sistema de Detecção de Equivalência de Disciplinas

# Notebook de verificação da similaridade entre ementas de disciplinas

## Introdução:
O estudante da UFABC passa muito tempo já em sua graduação para terminar as disciplinas do seu BI e do pós-BI e isso afeta principalmente os alunos de cursos mais concorridos como é o de computação, em que as disciplinas podem ter mais de 150% de requisição. Tendo em vista isso, na UFABC temos dois processos já estruturados que é o processo de covalidação e de equivalência, normatizados nas resoluções ConsEPE nº 157/2013 e CG Nº 023/2019 respectivamente. Covalidação é um processo interno da UFABC que é basicamente para a transição de projetos pedagógicos, de forma que o(a) estudante consegue integralizar o curso em um PPC antigo com disciplinas novas e a equivalência é um processo que uma disciplina de fora pode ter alguma similaridade de uma disciplina da ufabc e do curso que você quer se formar. Assim está presente na Resolução CG Nº 023/2019 o seguinte:

>Art. 4º Consistem em requisitos para a dispensa por equivalência, para disciplinas
cursadas no Brasil:

>> I. a carga horária total da disciplina cursada deve ser igual ou maior à carga horária da que se pede equivalência;

>> II. o conteúdo da disciplina cursada deve ser compatível e correspondente a, no mínimo, 75% (setenta e cinco por cento) do conteúdo daquela de que se pede equivalência, considerando-se teoria e prática, quando for o caso. 

>>>Parágrafo único: Excepcionalmente, e mediante justificativa, a coordenação de curso pode autorizar equivalências que cumpram parcialmente estes requisito

Utilizando essa base das normativas e os catálogos de disciplinas da UFABC, objetivamos gerar um sistema de pré-avaliação de disciplinas com alta chance de convalidação, reduzindo o espaço de busca dos técnicos responsaveis pela aprovação de pares de disciplinas com convalidação e garantindo a inclusão de todas as disciplinas na análise. Os benefício desta proposta são a economia de recursos, maior integralização de diferentes PPCs e promove efetivamente a interdisciplinaridade, fundamento da UFAB, uma vez que os cursos poderiam ofertar a mesma disciplina com diferentes enfoques no mesmo quadrimestre, melhorando a qualidade do ensino. Os recursos poupados são tanto na carga de trabalho dos técnicos quanto recursos computacionais (realizada manualmente, ess tarefa tem um complexidade O(n^2)), requerindo análise manual apenas das disciplinas que se encaixam nos critérios das resoluções ConsEPE nº 157/2013 e CG Nº 023/2019 e com alta probabilidade de validação. 

Dessa forma propomos uma análise de similaridade semantica e relacional entre as ementas, os pré-requisitos das disciplinas da UFABC e dos valores de TPEI, para fim de pré-avaliação de equivalências entre as diciplinas da universidade cumprindo com o Art. 4° da resolução CG Nº 023/2019. Assim. em disciplinas que verificarmos que existe uma similaridade equivalente a de pares de disciplinas atualmente validadas (similaridade maior ou igual a 75% e que cumpre a quantidade de creditos da outra disciplina), poderemos gerar listas de pares para aprovação manual em ordem de similaridde. O benefício proporcionado pela maior integração do sistema é difícil de mensurar mas, além de afetar todes es envolvides no processo discente, o alinhamento dessa solução com os sistemas vigentes da universidade amplificam seu impacto e condições em que ele se manifesta.

## Objetivos
1. Fazer uma proposta de equivalência interna de disciplinas a partir da análise de similaridade semântica entre as ementas das disciplinas da UFABC da Graduação e da quantidade de TPEI que elas tem
2. Reduzir o espaço de busca de O(n²) para aproximadamente O(n*log(n)) a partir da aplicação de filtros de TPEI para quantidade de créditos das disciplinas, de forma só comparar as diciplinas que tem a quantidade de créditos iguais

## Métodos
1. Embedding semaântico do conteúdo dos Objetivos e Ementas das disciplinas utilizando BERTimbau e o cálculo de similaridades de cosseno
2. Grafo relacional dos Pré-Requisitos entre disciplinas e cálculo de similçaridade utiliando distâncias de jaccard
3. Otimização dos hiperparâmetros dos filtros de redução do espaço de busca utilizando modelos de árvores aleatórias 

## 1. Imports e Configuração

In [1]:
# Realiza imports necessários
# pip install -r requirements.txt

import csv
import numpy as np
import pandas as pd
import networkx as nx
import unicodedata
import requests
from io import StringIO
from node2vec import Node2Vec
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sentence_transformers import SentenceTransformer, models
import plotly.graph_objects as go


ModuleNotFoundError: No module named 'node2vec'

## 2. Carregamento de Dados

In [ ]:
# Baixa o catálogo da UFABC direto do Repositório no GitHub
def carregar_catalogo():
    url = "https://raw.githubusercontent.com/angeloodr/disciplinas-ufabc/main/catalogo_disciplinas_graduacao_2024_2025.tsv"
    print("🔄 Baixando catálogo de disciplinas...")
    resp = requests.get(url)
    resp.raise_for_status()
    df = pd.read_csv(StringIO(resp.text), sep='\t')

    # Normaliza os nomes das colunas
    df.columns = [
        normalize_str(col).upper().replace(' ', '_')
        for col in df.columns
    ]
    print("✅ Download bem-sucedido!")
    print("📝 Colunas disponíveis:", df.columns.tolist())
    return df


## 3. Preparação de Dados

In [ ]:
# Extrai dados de TPEI
#def parse_tpei(tpei_str):
#    # Converts "2-0-0-2" to dict with numeric values
#    values = tpei_str.split('-')
#    return {'T': int(values[0]), 'P': int(values[1]), 
#            'E': int(values[2]), 'I': int(values[3])}
#
# catalog_df['TPEI_parsed'] = catalog_df['TPEI'].apply(parse_tpei)

def normalize_text(text):
    if pd.isna(text):
        return []

    norm = unicodedata.normalize('NFKD', str(text))
    norm = norm.encode('ASCII', 'ignore').decode('utf-8')
    norm = re.sub(r'[^\w\s]', '')
    norm = norm.lower().strip()
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in text.split() if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)


def extract_tpei(tpei_str):
    if pd.isna(tpei_str):
        return []
    
    values = tpei_str.split('-')
    return {
        'teoria': int(values[0]),
        'pratica': int(values[1]),
        'extensao': int(values[2]),
        'individual': int(values[3]),
        'total_creditos': int(values[0]) + int(values[1])  # T+P only
    }



def extract_prereq(recomendacao):
    if pd.isna(recomendacao) or recomendacao.strip() == '':
        return []

    prereqs = []
    for part in recomendacao.split(';'):
        part = part.strip()
        if part:
            prereqs.append(part)
    return prereqs

def extract_cod(sigla):
    if pd.isna(sigla):
        return []
    return normalize_text(sigla)

def create_allfeats(df):
    df = df.copy()

    # TPEI
    tpei_feats = df['TPEI'].apply(extract_tpei)
    df['teoria'] = tpei_feats.apply(lambda x: x['teoria'])
    df['pratica'] = tpei_feats.apply(lambda x: x['pratica'])
    df['extensao'] = tpei_feats.apply(lambda x: x['extensao'])
    df['individual'] = tpei_feats.apply(lambda x: x['individual'])
    df['total_creditos'] = tpei_feats.apply(lambda x: x['total_creditos'])

    # Ementa
    df['ementa_norm'] = df['EMENTA'].apply(normalize_text)
    df['objetivos_norm'] = df['OBJETIVOS'].apply(normalize_text)

    # Pré-requisitos
    df['prerequisites'] = df['RECOMENDAÇÃO'].apply(extract_prereq)
    df['num_prerequisites'] = df['prerequisites'].apply(len)
    df['codigo'] = df['SIGLA'].apply(extract_cod)

    return df

## 4. Filtro TPEI

In [ ]:
# for pair in pairs
#   se total creditos[current] > total_creditos[comp]
#       remove.pair
#   else
#       tpei_dif = comp - current
#
# Se os créditos totais da disciplina sendo comparada for maior do que da disciplina sendo analisada
# Remove par da analise
# Log de pares excluidos?
# Senão calcula diferença de créditos entre matérias
# Return lista de pares e tpei_dif deles

## 5. Filtro de Pré-requisitos

In [ ]:
# Create DiGraph from prerequisite relationships
# Generate node2vec embeddings for disciplines
# Calculate cosine distance between discipline embeddings
# Apply prerequisite similarity threshold
# Filter pairs based on prerequisite similarity

#  Constrói grafo com arestas de pré-requisito com base na coluna RECOMENDACAO
def construir_grafo(df: pd.DataFrame) -> nx.DiGraph:
    print("📌 Construindo grafo de pré-requisitos...")
    G = nx.DiGraph()
    total_arestas = 0

    # Cria um dicionário de nome normalizado → sigla
    mapping = {
        normalizar_nome(row['DISCIPLINA']): row['SIGLA']
        for _, row in df.iterrows()
        if pd.notna(row['SIGLA'])
    }

    # Adiciona todos os nós com metadados
    for _, row in df.iterrows():
        sigla = row['SIGLA']
        nome = row['DISCIPLINA']
        if pd.isna(sigla): 
            continue
        G.add_node(sigla, nome=nome)

    # Adiciona as arestas baseadas nas recomendações
    for _, row in df.iterrows():
        curso = row['SIGLA']
        if pd.isna(curso):
            continue

        recs = row.get('RECOMENDACAO', '')
        if pd.isna(recs) or not isinstance(recs, str):
            continue

        for rec in recs.split(';'):
            rec = rec.strip()
            if not rec:
                continue
            rec_norm = normalizar_nome(rec)
            if rec_norm in mapping:
                prereq = mapping[rec_norm]
                if not G.has_edge(prereq, curso):
                    G.add_edge(prereq, curso, tipo='pre_requisito')
                    total_arestas += 1

    print(f"✅ Grafo criado com {G.number_of_nodes()} nós e {total_arestas} arestas.")
    return G

## 6. Cálculo de Score TPEI + Pré-requisitos

In [ ]:
# Prepara matriz de caracteristicas
def prepare_combined_features(discipline_pairs, tpei_diff, prereq_similarities):
    features = []
    for pair in discipline_pairs:
        prereq_sim = prereq_similarities.get(pairs, 0.0)
        features.append([tpei_diff, prereq_sim])
    return np.array(features)


# Treina CatBoost usando labeled_data
# Código da Larissa



# Otimiza limiar
# from sklearn.metrics import precision_recall_curve
y_pred_proba = model_catboost.predict_proba(X_val)[:,1]

precision, recall, thresholds = precision_recall_curve(y_val, y_pred_proba)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]

# Calcula score de cada par
all_features = prepare_combined_features(candidate_pairs, tpei_diff, prereq_similarities)
combined_scores = model_catboost.predict_proba(all_features)[:. 1]

# Aplica filtro em cada par
filtered_pairs = [
    pair for pair, score in zip(candidate_pairs, combined_scores)
    if score >= optimal_threshold
]

## 7. Filtro de Ementa

In [ ]:
# Load BERTimbau pre-trained model
# Generate embeddings for ementa texts
# Apply node2vec to ementa embeddings
# Calculate cosine distance between ementa embeddings
# Filter based on semantic similarity threshold

import re

def sao_variantes_simples(nome1, nome2):
    base1 = re.sub(r'[\s\-]*(laboratório|[a-zA-Z]{1,3}|[ivxIVX0-9]{1,4})\s*$', '', nome1.strip(), flags=re.IGNORECASE)
    base2 = re.sub(r'[\s\-]*(laboratório|[a-zA-Z]{1,3}|[ivxIVX0-9]{1,4})\s*$', '', nome2.strip(), flags=re.IGNORECASE)
    return base1.lower() == base2.lower()

def similariry_between_DISCIPLINA(cosine_sim, similarity_threshold = 0.75):
    # Encontrar pares com similaridade ≥ 75%
    similar_pairs = []
    n = len(df)
    for i in range(n):
        for j in range(i+1, n):  # Evitar duplicatas (i, j) e (j, i)
            nome_i = df.iloc[i]['DISCIPLINA']
            nome_j = df.iloc[j]['DISCIPLINA']
            if sao_variantes_simples(nome_i, nome_j):
                continue  # pula se são versões quase idênticas da mesma disciplina
            if cosine_sim[i, j] >= similarity_threshold:
                similar_pairs.append((nome_i, nome_j, cosine_sim[i, j]))

    # Exibir resultados
    for pair in similar_pairs:
        print(f"Disciplinas similares: {pair[0]} e {pair[1]} (Similaridade: {pair[2]:.2f})")
    
    return similar_pairs

model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
embeddings = model.encode(df['EMENTA_PREPROCESSED'].tolist())
cosine_sim_bert = cosine_similarity(embeddings, embeddings)
sim_pair = similariry_between_DISCIPLINA(cosine_sim_bert)

model_name = "neuralmind/bert-base-portuguese-cased"

# Criar modelo SBERT a partir do BERT
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
embeddings = model.encode(df['EMENTA_PREPROCESSED'].tolist())
cosine_sim_sbert = cosine_similarity(embeddings, embeddings)
sim_pair = similariry_between_DISCIPLINA(cosine_sim_sbert)


## 8. Cálculo de Score Final

In [ ]:
# Prepara caracteristicas
def prepare_final_features(pairs, tpei_dif, prereq_sim, ementa_sim):
    features = []
    for pair in pairs:
        prereq = prereq_sim.get(pair, 0.0)
        ementa = ementa_sim(pair, 0.0)
        features.append([tpei_dif, prereq, ementa])
    return np.array(features)

# Treino SVM com Kernel RBF
# Codigo da Larissa

# Calcula limiar ótimo
y_prob = svm_model.predict_proba(X_scaled)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_final, y_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
final_threshold = thresholds[np.argmax(f1_scores)]

all_final_features = prepare_final_features(
    filtered_pairs, tpei_diff, prereq_sim, ementa_sim
)
all_scaled = scaler.transform(all_final_features)
final_scores = svm_model.predict_proba(all_scaled)[:, 1]

#Lista final de pares
equivalent_pairs = [
    (pair, score) for pair, score in zip(filtered_pairs, final_scores)
    if score >= final_threshold
]

## 9. Explicação dos Resultados

In [ ]:
# import shap
explainer = shap.KernelExplainer(
    svm_model.predict_proba,
    X_scaled,
    link="logit"
)

shap_values = explainer.shap_values(all_scaled)

if isinstance(shap_values, list):
    shap_values = shap_values[1]

feature_names = ['Diferença de créditos', 'Similaridade de pré-requisito', 'Similaridade de ementa']
importance = np.abs(shap_values).mean(0)

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
}).sort_values('importance', ascending=False)

def explain_prediction(pair_index, features, shap_values):
    feature_values = features[pair_index]
    shap_value = shap_values[pair_index]
    
    explanation = []
    for i, (name, value, impact) in enumerate(zip(feature_names, feature_values, shap_value)):
        direction = "positive" if impact > 0 else "negative"
        explanation.append(f"{name}: {value:.3f} ({direction} impact: {abs(impact):.3f})")
    
    return "\n".join(explanation)

# Generate explanations for all equivalent pairs
explanations = []
for i, (pair, score) in enumerate(equivalent_pairs):
    explanation = f"Pair: {pair[0]} - {pair[1]}\n"
    explanation += f"Equivalence Score: {score:.3f}\n"
    explanation += "Feature Contributions:\n"
    explanation += explain_prediction(i, all_final_features, shap_values)
    explanations.append(explanation)
# Initialize SHAP explainer for SVM model
# Calculate SHAP values for each prediction
# Generate feature importance rankings
# Create individual prediction explanations
# Prepare text explanations for results

## 10. Visualizações dos Resultados

### 10.1 Matriz de visualização

In [ ]:
# Criar grafo
G = nx.Graph()

# Adicionar nós (disciplinas)
for sigla in df['DISCIPLINA'].unique():
    G.add_node(sigla)

# Modificar a criação de arestas
for pair in similar_pairs:
    disciplina_a, disciplina_b, similarity = pair
    if similarity >= 0.8:  # Ajuste o threshold aqui
        G.add_edge(disciplina_a, disciplina_b, weight=similarity)

        # Calcular posições dos nós
        pos = nx.kamada_kawai_layout(G, weight='weight')  # Usa o peso (similaridade) para organizar

        # Criar traços para arestas
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])  # None para separar linhas
            edge_y.extend([y0, y1, None])
        
        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line= dict(width=1, color='#888'),  # Espessura e cor das arestas
            hoverinfo='none',
            mode='lines')

# Criar traços para nós
node_x = []
node_y = []
node_text = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)  # Texto ao passar o mouse

node_trace = go.Scatter(
  x=node_x, y=node_y,
  mode='markers+text',
  text=node_text,
  textposition="top center",
  hoverinfo='text',
  marker=dict(
      showscale=True,
      colorscale='YlGnBu',
      size=15,
      color=[],  # Pode ser usado para codificar cores por comunidade
      line=dict(width=2, color='black'))
)

# Criar figura
fig = go.Figure(data=[edge_trace, node_trace],
    layout=go.Layout(
        showlegend=False,
        hovermode='closest',
        margin=dict(b=0, l=0, r=0, t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
)

# Adicionar interatividade (exibir sigla ao passar o mouse)
fig.update_traces(textposition='top center', hoverinfo='text')

from networkx.algorithms import community
communities = community.greedy_modularity_communities(G)
# Atribuir cores diferentes a cada comunidade

fig.show()

# Create DiGraph visualization showing prerequisite relationships
# Color nodes based on equivalence status
# Highlight connected discipline pairs
# Add node labels and edge weights
# Save graph visualization

### 10.2 Visualização por mapa de calor

In [ ]:
# Criar matriz de similaridade como DataFrame
similarity_df = pd.DataFrame(cosine_sim_bert, index=df['DISCIPLINA'], columns=df['DISCIPLINA'])

# Plotar heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_df, cmap='YlGnBu', annot=False, mask=(similarity_df < 0.5))
plt.title('Mapa de Calor de Similaridade entre Ementas')
plt.show()


# Generate SHAP summary plot for feature importance
# Create dependence plots for key features
# Visualize force plots for individual predictions
# Export plots for presentation

### 10.3 Matrizes de Confusão

In [ ]:
# Calculate confusion matrix for model performance
# Create heatmap visualization of confusion matrix
# Add precision, recall, and F1 scores
# Generate performance metrics report

## 11. Exportação de Resultados

In [ ]:
# Save filtered results to TSV file
# Export model performance metrics
# Save visualizations in specified formats
# Generate final summary report